In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:     path_main  = file.read()
with open('../path_gwls.txt', 'r') as file:     path_gwls  = file.read()
with open('../path_EUR-11.txt', 'r') as file:   path_eur11 = file.read()
    
dir_CORDEX  = f'{path_main}Data/EURO-CORDEX/'
dir_GWL     = f'{path_gwls}cmip5_all_ens/'
dir_EMT     = f'{path_main}Data/EURO-CORDEX/EMT/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_scripts = f'{path_main}/Scripts/'
dir_sftlf   = f'{path_eur11}/historical/sftlf/'
dir_out     = f'{path_main}Data/Plot_preparation/HWMId/'
if not os.path.exists(dir_out): os.mkdir(dir_out)


## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade', 'Zagreb',
          'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam', 'London', 'Dublin',
          'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk', 'Vilnius', 'Riga', 'Moscow',
          'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)

#Define HSIs
HSI_names = ['TX', 'TN']

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp26'] = []
all_models['rcp85'] = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP26.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp26'].append(eval(line[:-1]))
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(eval(line[:-1]))

#Read warming levels
fname = dir_GWL + 'cmip5_warming_levels_all_ens_1850_1900_no_bounds_check.yml'
with open(fname, 'r') as file:
    GWL_data = yaml.safe_load(file)

#Define warming levels
GWL_levels = ['10', '20', '30']

#Define out_name
out_name = '_3x3'
# out_name = ''


## Read land-sea mask

In [ ]:
N = 3

#Initialize dictionary
data_LSM = dict()

#Loop over all models
for model in all_models['rcp85']:
    
    #Get file names
    fnames = [file for file in os.listdir(dir_sftlf) if model[0] in file and model[1] in file]
    
    #Define file names for certain models, for which no LSM exists
    if model[1]=='IPSL-WRF381P':
        fnames = ['sftlf_EUR-11_CNRM-CERFACS-CNRM-CM5_historical_r0i0p0_IPSL-WRF381P_v2-ADAPTED-FROM-OLDER-VERSION_fx.nc']
    if model[0]=='ICHEC-EC-EARTH' and model[1]=='CLMcom-ETH-COSMO-crCLIM-v1-1':
        fnames = ['sftlf_EUR-11_MPI-M-MPI-ESM-LR_historical_r0i0p0_CLMcom-ETH-COSMO-crCLIM-v1-1_v1_fx.nc']
    if model[0]=='IPSL-IPSL-CM5A-MR' and model[1]=='DMI-HIRHAM5':
        fnames = ['sftlf_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_DMI-HIRHAM5_v1_fx.nc']    
    
    #Get file name
    if len(fnames)>=1:
        fname = fnames[0]
    else:
        sys.exit('File name not defined')
    
    #Read data
    data_sftlf = xr.open_dataset(dir_sftlf + fname)
 
    #Convert LSM data to %
    if data_sftlf.sftlf.max()<50:
        data_sftlf['sftlf'] = 100 * data_sftlf['sftlf']

    #Convert longitude from [0, 360] to [-180, 180]
    if 'longitude' in data_sftlf.coords:  lat_name, lon_name = 'latitude', 'longitude'
    elif 'lon' in data_sftlf.coords:      lat_name, lon_name = 'lat', 'lon'
    if data_sftlf[lon_name].max()>180:
        data_sftlf[lon_name] = data_sftlf[lon_name].where(data_sftlf[lon_name]<180, ((data_sftlf[lon_name] + 180) % 360) - 180)
    
    #Loop over cities
    for city in cities:
        
        #Get lat and lon of city
        lat_sel, lon_sel = city_coords[city]

        #Find grid point closest to city
        loc_city = (np.abs(data_sftlf[lon_name] - lon_sel)) + (np.abs(data_sftlf[lat_name] - lat_sel))
        ind_city = np.unravel_index(np.argmin(loc_city.values), loc_city.shape)

        #Select NxN box around grid point
        N1 = int(N/2 - 0.5)
        N2 = int(N/2 + 0.5)
        lat_rng  = slice(ind_city[0] - N1, ind_city[0] + N2)
        lon_rng  = slice(ind_city[1] - N1, ind_city[1] + N2)
        if 'rlat' in data_sftlf.dims:   data_sftlf_city = data_sftlf.isel(rlat=lat_rng, rlon=lon_rng)
        elif 'x' in data_sftlf.dims:    data_sftlf_city = data_sftlf.isel(y=lat_rng, x=lon_rng)
        
        #Save in dictionary
        data_LSM[model[0] + '_' + model[1] + '_' + city] = data_sftlf_city.sftlf


## Prepare EMT data

In [ ]:
#Define folders
dir_HWMId = dir_CORDEX + 'HWMId' + out_name + '/'
dir_save  = dir_out + 'EURO-CORDEX' + out_name + '/'

#Define RCP
RCP = 'rcp85'

#Define quantiles
quantiles = ['Q50', 'Q90']

#Define changes in EMT relative to 1981-2010
EMT_change     = np.array([1.0, 2.0, 3.0])
EMT_change_str = ['1.0K', '2.0K', '3.0K']

#Define minimum fraction of land required on each grid point
land_fractions = [0, 25, 50, 75, 100]

#Loop over cities
for HSI in HSI_names:
    
    print(HSI)

    #Combine model names to one string
    mod_names = ["_".join(model) for model in all_models[RCP]]

    #Create empty numpy array and define coordinates
    if out_name=='_3x3':
        da_empty = np.zeros((len(all_models[RCP]), len(cities), len(EMT_change), len(quantiles), len(land_fractions), 2)) * np.NaN
        coords={'model':        ('model', mod_names),
                'city':         ('city', cities),
                'EMT_change':   ('EMT_change', EMT_change_str),
                'quantile':     ('quantile', quantiles),
                'land_frac':    ('land_frac', land_fractions),
                'range_3x3box': ('range_3x3box', ['min', 'max'])}
    else:
        da_empty = np.zeros((len(all_models[RCP]), len(cities), len(EMT_change), len(quantiles))) * np.NaN
        coords={'model':      ('model', mod_names),
                'city':       ('city', cities),
                'EMT_change': ('EMT_change', EMT_change_str),
                'quantile':   ('quantile', quantiles)}    
    
    #Create empty data array for storing SREX-averaged heat indices
    data_coll = xr.Dataset(coords=coords)
    data_coll['HWMID'] = (coords.keys(), da_empty)

    #Loop over cities
    for city in cities:
        
        #Select folder
        if out_name=='':  dir_data = dir_HWMId + 'HWMId-' + HSI + '/'    
        else:             dir_data = dir_HWMId + city + '/' + RCP + '/'
                
        #Loop over models
        for i1, model in enumerate(all_models[RCP]):

            #Read data
            fname_CORDEX = [file for file in os.listdir(dir_data) if HSI in file and RCP + '_' in file and model[0] in file and model[1] in file and model[2] in file]
            if len(fname_CORDEX)!=1:  sys.exit('File is not unique')
            data_CORDEX = xr.open_dataset(dir_data + fname_CORDEX[0])              
            
            #Select city
            if out_name=='':  data_CORDEX = data_CORDEX.sel(city=city)
            
            #Read European mean temperature (EMT)
            files_EMT = sorted([dir_EMT + file for file in os.listdir(dir_EMT) if "_".join(model) in file and 'EMT_' in file])
            data_EMT  = xr.concat((xr.open_dataset(file) for file in files_EMT), dim='time')
            
            #Calculate EMT relative to 1981-2010 and calculate 20-year means
            dataEMT_ref = data_EMT.sel(time=slice('1981', '2010')).mean('time')
            dataEMT_rel = data_EMT - dataEMT_ref
            dataEMT_20y = dataEMT_rel.rolling(time=20, center=True).mean()

            #Loop over selected EMT levels
            for dEMT, dEMT_str in zip(EMT_change, EMT_change_str):

                #Identify 20-year period in wich level is reached for first time
                ind  = np.where(dataEMT_20y.tas>dEMT)[0][0]
                central_year = dataEMT_20y.isel(time=ind).time.dt.year
                start_year   = int(central_year - 20 / 2)
                end_year     = int(central_year + (20 / 2 - 1))
                years_sel    = slice(str(start_year), str(end_year))
                
                if end_year>2099:
                    print(model)
                    print(end_year)
                
                #Calculate median and 90th quantile
                data_Q50 = data_CORDEX.HWMID.sel(time=years_sel).median('time')
                data_Q90 = data_CORDEX.HWMID.sel(time=years_sel).quantile(0.90, dim='time')
                
                if out_name=='_3x3':
                    
                    #Read land fraction for each model and city
                    land_frac = data_LSM[model[0] + '_' + model[1] + '_' + city]

                    #Rename land fraction for MPI-WRF combination
                    if (model[0]=='MPI-M-MPI-ESM-LR') and (model[1]=='IPSL-WRF381P'):
                        land_frac = land_frac.rename({'rlat': 'y', 'rlon': 'x'})                       

                    #Get coordinate names
                    if 'longitude' in land_frac.coords:  lat_name1, lon_name1 = 'latitude', 'longitude'
                    elif 'lon' in land_frac.coords:      lat_name1, lon_name1 = 'lat', 'lon'                        
                    if 'rlon' in land_frac.coords:       lat_name2, lon_name2 = 'rlat', 'rlon'
                    elif 'x' in land_frac.coords:        lat_name2, lon_name2 = 'y', 'x'   

                    #Check if coordinates agree and reindex if they are slightly shifted
                    check1a = np.sum(np.abs(land_frac[lat_name1].values - data_Q50[lat_name1].values))
                    check1b = np.sum(np.abs(land_frac[lon_name1].values - data_Q50[lon_name1].values))
                    check2a = np.sum(np.abs(land_frac[lat_name2].values - data_Q50[lat_name2].values))
                    check2b = np.sum(np.abs(land_frac[lon_name2].values - data_Q50[lon_name2].values))
                    if (check1a>0.001) or (check1b>0.001):
                        sys.exit('Coordinates between data and LSM do not agree')
                    if (check2a!=0) or (check2b!=0):
                        land_frac[lon_name2] = data_Q50[lon_name2]
                        land_frac[lat_name2] = data_Q50[lat_name2]                  
                    
                    #Loop over different minimum land fractions
                    for land_min in land_fractions:

                        #Apply land-sea mask
                        data_Q90_land = data_Q90.where(land_frac>=land_min)
                        data_Q50_land = data_Q50.where(land_frac>=land_min)
                        
                        #Test that final array still has all entries
                        if data_Q90_land.shape!=(3,3):  sys.exit('bla')
                            
                        #Put data in array
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "quantile": 'Q50', "land_frac": land_min, 'range_3x3box': 'min'}] = data_Q50_land.min()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "quantile": 'Q50', "land_frac": land_min, 'range_3x3box': 'max'}] = data_Q50_land.max()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "quantile": 'Q90', "land_frac": land_min, 'range_3x3box': 'min'}] = data_Q90_land.min()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "quantile": 'Q90', "land_frac": land_min, 'range_3x3box': 'max'}] = data_Q90_land.max()

                else:
                    
                    #Put data in array
                    data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "quantile": 'Q50'}] = data_Q50
                    data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "quantile": 'Q90'}] = data_Q90

    #Save data in file
    fname_out = dir_save + 'HWMId-' + HSI + '_' + RCP + '_EMT.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
  

## Prepare GWL data

In [ ]:
#Define folders
dir_HWMId = dir_CORDEX + 'HWMId' + out_name + '/'
dir_save  = dir_out + 'EURO-CORDEX' + out_name + '/'

#Define RCP
RCP = 'rcp85'

#Define quantiles
quantiles = ['Q50', 'Q90']

#Define minimum fraction of land required on each grid point
land_fractions = [0, 25, 50, 75, 100]

#Loop over cities
for HSI in HSI_names:
    
    print(HSI)

    #Combine model names to one string
    mod_names = ["_".join(model) for model in all_models[RCP]]

    #Create empty numpy array and define coordinates
    if out_name=='_3x3':
        da_empty = np.zeros((len(all_models[RCP]), len(cities), len(GWL_levels), len(quantiles), len(land_fractions), 2)) * np.NaN
        coords={'model':        ('model', mod_names),
                'city':         ('city', cities),
                'GWL_level':    ('GWL_level', GWL_levels),
                'quantile':     ('quantile', quantiles),
                'land_frac':    ('land_frac', land_fractions),
                'range_3x3box': ('range_3x3box', ['min', 'max'])}
    else:
        da_empty = np.zeros((len(all_models[RCP]), len(cities), len(GWL_levels), len(quantiles))) * np.NaN
        coords={'model':     ('model', mod_names),
                'city':      ('city', cities),
                'GWL_level': ('GWL_level', GWL_levels),
                'quantile':  ('quantile', quantiles)}    
    
    #Create empty data array for storing SREX-averaged heat indices
    data_coll = xr.Dataset(coords=coords)
    data_coll['HWMID'] = (coords.keys(), da_empty)
    
    #Loop over cities
    for city in cities:
        
        #Select folder
        if out_name=='':  dir_data = dir_HWMId + 'HWMId-' + HSI + '/'    
        else:             dir_data = dir_HWMId + city + '/' + RCP + '/'
                
        #Loop over models
        for i1, model in enumerate(all_models[RCP]):

            #Get name of CMIP5 driving model
            if ('CNRM-CERFAC' in model[0]) or ('CSIRO-QCCCE' in model[0]) or ('MPI-M' in model[0]) or ('NOAA-GFDL' in model[0]):
                mod_CMIP5 = '-'.join(model[0].split('-')[2:])
            else:
                mod_CMIP5 = '-'.join(model[0].split('-')[1:])
            
            #Read data
            fname_CORDEX = [file for file in os.listdir(dir_data) if HSI in file and RCP + '_' in file and model[0] in file and model[1] in file and model[2] in file]
            if len(fname_CORDEX)!=1:  sys.exit('File is not unique')
            data_CORDEX = xr.open_dataset(dir_data + fname_CORDEX[0])              
            
            #Select city
            if out_name=='':  data_CORDEX = data_CORDEX.sel(city=city)
            
            #Read time periods when certain global warming levels (GWL) are reached
            time_GWL = dict()
            for level in GWL_levels:
                data_level = GWL_data['warming_level_' + level]
                entry_sel = [entry for entry in data_level if entry['model']==mod_CMIP5 and entry['exp']==RCP and entry['ensemble']==model[2]]
                
                #Select data in time period when GWL is reached
                time_GWL = slice(str(entry_sel[0]['start_year']), str(entry_sel[0]['end_year']))
                
                #Calculate median and 90th quantile
                data_Q50 = data_CORDEX.HWMID.sel(time=time_GWL).median('time')
                data_Q90 = data_CORDEX.HWMID.sel(time=time_GWL).quantile(0.90, dim='time')
                
                if out_name=='_3x3':
                    
                    #Read land fraction for each model and city
                    land_frac = data_LSM[model[0] + '_' + model[1] + '_' + city]

                    #Rename land fraction for MPI-WRF combination
                    if (model[0]=='MPI-M-MPI-ESM-LR') and (model[1]=='IPSL-WRF381P'):
                        land_frac = land_frac.rename({'rlat': 'y', 'rlon': 'x'})                       

                    #Get coordinate names
                    if 'longitude' in land_frac.coords:  lat_name1, lon_name1 = 'latitude', 'longitude'
                    elif 'lon' in land_frac.coords:      lat_name1, lon_name1 = 'lat', 'lon'                        
                    if 'rlon' in land_frac.coords:       lat_name2, lon_name2 = 'rlat', 'rlon'
                    elif 'x' in land_frac.coords:        lat_name2, lon_name2 = 'y', 'x'   

                    #Check if coordinates agree and reindex if they are slightly shifted
                    check1a = np.sum(np.abs(land_frac[lat_name1].values - data_Q50[lat_name1].values))
                    check1b = np.sum(np.abs(land_frac[lon_name1].values - data_Q50[lon_name1].values))
                    check2a = np.sum(np.abs(land_frac[lat_name2].values - data_Q50[lat_name2].values))
                    check2b = np.sum(np.abs(land_frac[lon_name2].values - data_Q50[lon_name2].values))
                    if (check1a>0.001) or (check1b>0.001):
                        sys.exit('Coordinates between data and LSM do not agree')
                    if (check2a!=0) or (check2b!=0):
                        land_frac[lon_name2] = data_Q50[lon_name2]
                        land_frac[lat_name2] = data_Q50[lat_name2]                  
                    
                    #Loop over different minimum land fractions
                    for land_min in land_fractions:

                        #Apply land-sea mask
                        data_Q90_land = data_Q90.where(land_frac>=land_min)
                        data_Q50_land = data_Q50.where(land_frac>=land_min)
                        
                        #Test that final array still has all entries
                        if data_Q90_land.shape!=(3,3):  sys.exit('bla')                    

                        #Put data in array
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "GWL_level": level, "quantile": 'Q50', "land_frac": land_min, 'range_3x3box': 'min'}] = data_Q50_land.min()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "GWL_level": level, "quantile": 'Q50', "land_frac": land_min, 'range_3x3box': 'max'}] = data_Q50_land.max()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "GWL_level": level, "quantile": 'Q90', "land_frac": land_min, 'range_3x3box': 'min'}] = data_Q90_land.min()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "GWL_level": level, "quantile": 'Q90', "land_frac": land_min, 'range_3x3box': 'max'}] = data_Q90_land.max()
                else:
                        
                    #Put data in array
                    data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "GWL_level": level, "quantile": 'Q50'}] = data_Q50
                    data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "GWL_level": level, "quantile": 'Q90'}] = data_Q90
                
    #Save data in file
    fname_out = dir_save + 'HWMId-' + HSI + '_' + RCP + '_GWL.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
  

## Prepare time period data

In [ ]:
#Define folders
dir_HWMId = dir_CORDEX + 'HWMId' + out_name + '/'
dir_save  = dir_out + 'EURO-CORDEX' + out_name + '/'

#Select time periods
time_periods = [[1981, 2010],
                [2036, 2065],
                [2070, 2099]]

#Define time string
time_strings = [str(time[0]) + '-' + str(time[1]) for time in time_periods]

#Define RCPs
RCP = 'rcp85'

#Define quantiles
quantiles = ['Q50', 'Q90']

#Define minimum fraction of land required on each grid point
land_fractions = [0, 25, 50, 75, 100]

#Loop over cities
for HSI in HSI_names:

    print(HSI)

    #Combine model names to one string
    mod_names = ["_".join(model) for model in all_models[RCP]]

    #Create empty numpy array and define coordinates
    if out_name=='_3x3':
        da_empty = np.zeros((len(all_models[RCP]), len(cities), len(time_periods), len(quantiles), len(land_fractions), 2)) * np.NaN
        coords={'model':        ('model', mod_names),
                'city':         ('city', cities),
                'time_period':  ('time_period', time_strings),
                'quantile':     ('quantile', quantiles),
                'land_frac':    ('land_frac', land_fractions),
                'range_3x3box': ('range_3x3box', ['min', 'max'])}
    else:
        da_empty = np.zeros((len(all_models[RCP]), len(cities), len(time_periods), len(quantiles))) * np.NaN
        coords={'model':       ('model', mod_names),
                'city':        ('city', cities),
                'time_period': ('time_period', time_strings),
                'quantile':    ('quantile', quantiles)}    
    
    #Create empty data array for storing SREX-averaged heat indices
    data_coll = xr.Dataset(coords=coords)
    data_coll['HWMID'] = (coords.keys(), da_empty)

    #Loop over cities
    for city in cities:

        #Select folder
        if out_name=='':  dir_data = dir_HWMId + 'HWMId-' + HSI + '/'    
        else:             dir_data = dir_HWMId + city + '/' + RCP + '/'

        #Loop over models
        for i1, model in enumerate(all_models[RCP]):

            #Get name of CMIP5 driving model
            if ('CNRM-CERFAC' in model[0]) or ('CSIRO-QCCCE' in model[0]) or ('MPI-M' in model[0]) or ('NOAA-GFDL' in model[0]):
                mod_CMIP5 = '-'.join(model[0].split('-')[2:])
            else:
                mod_CMIP5 = '-'.join(model[0].split('-')[1:])

            #Read data
            fname_CORDEX = [file for file in os.listdir(dir_data) if HSI in file and RCP + '_' in file and model[0] in file and model[1] in file and model[2] in file]
            if len(fname_CORDEX)!=1:  sys.exit('File is not unique')
            data_CORDEX = xr.open_dataset(dir_data + fname_CORDEX[0])              

            #Select city
            if out_name=='':  data_CORDEX = data_CORDEX.sel(city=city)

            #Loop over time periods
            for time_period, time_string in zip(time_periods, time_strings):

                #Select data in time period when GWL is reached
                time_sel = slice(str(time_period[0]), str(time_period[1]))
                data_Q50 = data_CORDEX.HWMID.sel(time=time_sel).median('time')
                data_Q90 = data_CORDEX.HWMID.sel(time=time_sel).quantile(0.90, dim='time')
                
                if out_name=='_3x3':
                    
                    #Read land fraction for each model and city
                    land_frac = data_LSM[model[0] + '_' + model[1] + '_' + city]

                    #Rename land fraction for MPI-WRF combination
                    if (model[0]=='MPI-M-MPI-ESM-LR') and (model[1]=='IPSL-WRF381P'):
                        land_frac = land_frac.rename({'rlat': 'y', 'rlon': 'x'})                       

                    #Get coordinate names
                    if 'longitude' in land_frac.coords:  lat_name1, lon_name1 = 'latitude', 'longitude'
                    elif 'lon' in land_frac.coords:      lat_name1, lon_name1 = 'lat', 'lon'                        
                    if 'rlon' in land_frac.coords:       lat_name2, lon_name2 = 'rlat', 'rlon'
                    elif 'x' in land_frac.coords:        lat_name2, lon_name2 = 'y', 'x'   

                    #Check if coordinates agree and reindex if they are slightly shifted
                    check1a = np.sum(np.abs(land_frac[lat_name1].values - data_Q50[lat_name1].values))
                    check1b = np.sum(np.abs(land_frac[lon_name1].values - data_Q50[lon_name1].values))
                    check2a = np.sum(np.abs(land_frac[lat_name2].values - data_Q50[lat_name2].values))
                    check2b = np.sum(np.abs(land_frac[lon_name2].values - data_Q50[lon_name2].values))
                    if (check1a>0.001) or (check1b>0.001):
                        sys.exit('Coordinates between data and LSM do not agree')
                    if (check2a!=0) or (check2b!=0):
                        land_frac[lon_name2] = data_Q50[lon_name2]
                        land_frac[lat_name2] = data_Q50[lat_name2]                  
                    
                    #Loop over different minimum land fractions
                    for land_min in land_fractions:

                        #Apply land-sea mask
                        data_Q90_land = data_Q90.where(land_frac>=land_min)
                        data_Q50_land = data_Q50.where(land_frac>=land_min)
                        
                        #Test that final array still has all entries
                        if data_Q90_land.shape!=(3,3):  sys.exit('bla')                    

                        #Put data in array
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "time_period": time_string, "quantile": 'Q50', "land_frac": land_min, 'range_3x3box': 'min'}] = data_Q50_land.min()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "time_period": time_string, "quantile": 'Q50', "land_frac": land_min, 'range_3x3box': 'max'}] = data_Q50_land.max()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "time_period": time_string, "quantile": 'Q90', "land_frac": land_min, 'range_3x3box': 'min'}] = data_Q90_land.min()
                        data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "time_period": time_string, "quantile": 'Q90', "land_frac": land_min, 'range_3x3box': 'max'}] = data_Q90_land.max()
                else:
                    
                    #Put data in array
                    data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "time_period": time_string, "quantile": 'Q50'}] = data_Q50
                    data_coll.HWMID.loc[{"model": "_".join(model), "city": city, "time_period": time_string, "quantile": 'Q90'}] = data_Q90

    #Save data in file
    data_coll.to_netcdf(dir_save + 'HWMId-' + HSI + '_' + RCP + '_time-periods.nc')
